# 还原论文算例

In [1]:
from utils import *

A1 = torch.tensor([[-1, 0, 0], [0, -2, 2], [0, 0, -2]], dtype=torch.float32)
A2 = torch.tensor([[-3, 0, 0], [0, -4, 4], [0, 0, -6]], dtype=torch.float32)
A=(A1,A2)

print(GetCommonEigenInfo(A))


(tensor([[1.],
        [0.],
        [0.]]), tensor([[-1., -3.]]))


c:\Users\CTC\Desktop\temporary working library\Python\KoopmanCommonLyapunovFunction\utils.py:62: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Copy.cpp:250.)


In [2]:
temp = (A1.to('cuda:0'), A2.to('cuda:0'))
GetCommonEigenInfo(temp,
                   Blacklist=torch.tensor([[1], [0], [0]], device='cuda:0'),
                   device='cuda:0')


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [3]:
temp=ObtainInvariantMaximalFlag(A,device="cuda")
#temp=ObtainInvariantMaximalFlag(A)
temp.ComputeFlagInfo()


tensor([[1.],
        [0.],
        [0.]], device='cuda:0')
tensor([[-1., -3.]], device='cuda:0')
tensor([[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')


AttributeError: 'tuple' object has no attribute 'to'

原文中计算$\epsilon_{j}$的公式如下：

$$
\epsilon_{j} > \max_{\substack{i \in \{1, \cdots, m \} \\ j \in \{1, \cdots, j - 1 \}}} \epsilon_{k} \frac{(n - 1)^{2}}{4} \frac{\lvert \vec{v}_{k}^{*} A_{i} \vec{v}_{j} \rvert^{2}}{\lvert \operatorname{Re} (\lambda_{i,j}) \rvert \lvert \operatorname{Re} (\lambda_{i,k}) \rvert}
$$

定义计算$\epsilon_{j}$可能的值的函数如下


In [ ]:
import numpy as np


def EpsilonPossible(i, j, k, Dimension, EpsilonInputArray, v, A, Lambda):
    # i,j,k表示公式中的i,j,k下标，从1开始
    # EpsilonInputArray是计算时候输入的$\epsilon_{k}$的序列，在取完max后更新
    # A表示线性切换系统子系统的系统矩阵
    # $v_{j}$等表示求maximal invarient flag时对应的特征向量
    # $\lambda_{i,j}$表示共同的特征向量对应的特征值
    VkConjugateTransAiVj = v[:,k - 1].conj().T @ A[i - 1] @ v[:,j - 1]
    return EpsilonInputArray[k - 1] * np.power(
        Dimension - 1, 2) / 4 * np.power(
            np.linalg.norm(VkConjugateTransAiVj), 2) / (np.abs(
                Lambda[i - 1, j - 1].real) * np.abs(Lambda[i - 1, k - 1].real))


定义一个返回共同特征向量及对应特征值的函数

In [ ]:
import itertools


def GetCommonEigVecInfo(Subsystemset, Blacklist=None):
    # 论文都是两个子系统的例子，这里只考虑两个子系统的情况
    n = Subsystemset[0].shape[0]
    temp1 = np.linalg.eig(Subsystemset[0])
    temp2 = np.linalg.eig(Subsystemset[1])

    # 删除对应于Blacklist中元素的项
    if Blacklist != None:
        for i in range(Blacklist.shape[1]):
            print("i = ",i)
            for j in range(temp1[1].shape[1]):
                print("j = ",j)
                if np.array_equal(Blacklist[:, i], temp1[1][:, j]):
                    temp1 = (np.delete(temp1[0],
                                       j), np.delete(temp1[1], j, axis=1))
                if np.array_equal(Blacklist[:, i], temp2[1][:, j]):
                    temp2 = (np.delete(temp2[0],
                                       j), np.delete(temp2[1], j, axis=1))

    for i, j in itertools.product(range(n), range(n)):
        if np.array_equal(temp1[1][:, i], temp2[1][:, j]):
            return temp1[0][i], temp2[0][j], temp1[1][:, i]


In [ ]:
temp1=np.diag([1,2,3])
temp2=np.diag([1,2,4])

blacklisttemp=np.array([[1, 0], [0, 1], [0, 0]])

for i in range(blacklisttemp.shape[1]):
    print(i)

print(np.linalg.eig(temp1),"\n",np.linalg.eig(temp2))
GetCommonEigVecInfo((np.diag([1, 2, 3]), np.diag([1, 2, 4])),
                    np.array([[1, 0], [0, 1], [0, 0]]))


定义一个根据$V=\begin{bmatrix} \vec{v}_{1} & \vec{v}_{2} & \cdots & \vec{v}_{k} \end{bmatrix}$矩阵计算$P_{k}$的函数


In [ ]:
def GetPMatrix(Vmatrix):
    return Vmatrix @ np.linalg.inv(
        Vmatrix.conj().T @ Vmatrix) @ Vmatrix.conj().T


定义一个返回切换所有$\lambda$和$\vec{v}$的函数


In [ ]:
def GetInvariantMaximalFlagInfo(Subsystemset):
    # 论文都是两个子系统的例子，这里只考虑两个子系统的情况
    n = Subsystemset[0].shape[0]
    TempSubsysSet = Subsystemset

    # 初始化Lambda矩阵，$\lambda_{i,j}$对应于其元素Lambda[i-1,j-1]
    Lambda = np.zeros((2, n))

    for i in range(n):
        # 计算$\lambda_{1,i+1}$和$\lambda_{2,i+1}$和$\vec{v}_{i+1}$
        if i == 0:
            temp = GetCommonEigVecInfo(TempSubsysSet)
            V = temp[2].reshape(-1, 1)
        else:
            temp = GetCommonEigVecInfo(TempSubsysSet, V)
            V = np.append(V, temp[2].reshape(-1, 1), axis=1)

        Lambda[:, i] = temp[:2]

        print("i = ", i)

        print(temp)

        print("V =\n", V)

        # 更新TempSubsysSet
        Ptemp = GetPMatrix(V.reshape(n, -1))
        print("P =\n", Ptemp)
        TempSubsysSet = ((np.identity(n) - Ptemp) @ TempSubsysSet[0],
                         (np.identity(n) - Ptemp) @ TempSubsysSet[1])
        print("b")

    return Lambda, V


## Example 1

定义两个子系统

In [ ]:
A1 = np.array([[-1, 0, 0], [0, -2, 2], [0, 0, -2]])
A2 = np.array([[-3, 0, 0], [0, -4, 4], [0, 0, -6]])
A = (A1, A2)


In [ ]:
Lambda=np.zeros((2,3))
Lambda[:,0]=GetCommonEigVecInfo(A)[:2]
V=[]
V=np.append(V,GetCommonEigVecInfo(A)[2].reshape(-1,1)).reshape(3,-1)

V=np.append(V,V,axis=1)

V

In [ ]:
GetInvariantMaximalFlagInfo(A)